In [ ]:
# Set working directory (optional during development)
import os
os.chdir('/Users/sudishmakarki/My_project2')  # only if needed
print(" Working directory:", os.getcwd())

# Standard Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from prophet.diagnostics import cross_validation, performance_metrics
import scipy.stats as stats


# Custom Functions
from models.data_interpolation import (
    load_data,
    preprocess_data,
    split_train_test,
    generate_time_series_splits
)

from models.model_sarimax import (
    prepare_sarimax_data,
    check_stationarity,
    plot_acf_pacf,
    fit_sarimax_model,
    analyze_residual_spike,
    ljung_box_test,
    forecast_sarimax_model,
    identify_peak_hours_sarimax,
    evaluate_sarimax_metrics,
    rolling_forecast_sarimax,
    generate_future_forecast_sarimax,
    group_forecast_by_hour,
    
)

from models.model_sarimax import (
    sarimax_grid_search,
    retrain_sarimax_model,
    ljung_box_test_residuals,
    forecast_with_refined_sarimax,
    analyze_peak_hours_sarimax_refined,
    evaluate_refined_sarimax_metrics,
    rolling_forecast_sarimax_refined,
    forecast_future_sarimax_model_refined,
    future_forecast_by_hour_sarimax_refined
)

In [ ]:
# Load and preprocess
df = load_data('data/RestaurantData.csv')
df_clean = preprocess_data(df)

In [ ]:
# Split into train and test
restaurant_train, restaurant_test = split_train_test(df_clean, split_date='2022-01-01')
# Format for SARIMAX
train_series, test_series = prepare_sarimax_data(restaurant_train, restaurant_test)

In [ ]:
# --- Grid Search for Best SARIMAX Parameters ---
results_df, best_params = sarimax_grid_search(train_series)

In [ ]:
# --- Retrain the SARIMAX Model ---
best_order = best_params['order']
best_seasonal_order = best_params['seasonal_order']
results_sarimax_best = retrain_sarimax_model(train_series, best_order, best_seasonal_order)

In [ ]:
# --- Residual Diagnostics ---
residuals_refined = results_sarimax_best.resid

plt.figure(figsize=(12, 6))
plt.plot(residuals_refined)
plt.title("Residuals Over Time (Refined SARIMAX Model)")
plt.xlabel("Time")
plt.ylabel("Residuals")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 5))
sns.histplot(residuals_refined.dropna(), kde=True, color='blue')
plt.title("Histogram of Residuals (Refined SARIMAX Model)")
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
stats.probplot(residuals_refined.dropna(), dist="norm", plot=plt)
plt.title("Q-Q Plot of Residuals (Refined SARIMAX Model)")
plt.tight_layout()
plt.show()

In [ ]:
# --- Ljung-Box Test ---
ljung_box_test_residuals(residuals_refined)

In [ ]:
# --- Step 7: Forecast on Test Set ---
forecast_mean, forecast_ci = forecast_with_refined_sarimax(results_sarimax_best, test_series)

plt.figure(figsize=(12, 6))
plt.plot(train_series.index, train_series, label='Training Data', color='steelblue')
plt.plot(test_series.index, test_series, label='Actual Test Data', color='blue')
plt.plot(forecast_mean.index, forecast_mean, label='SARIMAX Forecast', color='red')
plt.fill_between(forecast_ci.index,
                 forecast_ci.iloc[:, 0],
                 forecast_ci.iloc[:, 1],
                 color='pink', alpha=0.3, label='Confidence Interval')
plt.title("SARIMAX Forecast vs Actual (Refined Model)")
plt.xlabel("Time")
plt.ylabel("CustomerCount")
plt.legend()
plt.show()

In [ ]:
# --- Step 8: Peak Hour Analysis ---
peak_hours_sr1, threshold, hourly_avg, forecast_peak, test_peak = analyze_peak_hours_sarimax_refined(forecast_mean, test_series)

# Display hourly average forecast nicely
display(hourly_avg.reset_index(name='Avg Forecast (yhat)').style.set_caption("Refined SARIMAX: Hourly Avg Forecast").background_gradient(cmap='YlOrRd'))  # warm colors for peak intensity

print("\nThreshold for Peak Hours:", threshold)
print("\nDynamically Identified Peak Hours:", peak_hours_sr1)

In [ ]:
# --- Step 9: Error Metrics ---
metrics_table = evaluate_refined_sarimax_metrics(test_series, forecast_mean, test_peak, forecast_peak)
refined_metrics_df = pd.DataFrame(metrics_table, columns=['Metric', 'Type', 'Value'])
display(refined_metrics_df.style.set_caption("Refined SARIMAX: Forecast Evaluation Metrics").background_gradient(cmap='Blues', subset=['Value']))


In [ ]:
# --- Step X: Rolling Forecast Evaluation (Refined SARIMAX) ---

rolling_overall_metrics, rolling_peak_metrics, roll_fcst, roll_actual, roll_fcst_peak, roll_actual_peak = rolling_forecast_sarimax_refined(
    train_series=train_series,
    test_series=test_series,
    best_order=best_order,
    best_seasonal_order=best_seasonal_order,
    peak_hours=peak_hours_sr1,  # from your peak hour analysis
    window_size=500,
    step=5,
    forecast_steps=1,
    max_points=50
)

rolling_metrics_ref1 = pd.DataFrame([
    ["MAE", "Overall", rolling_overall_metrics['MAE']],
    ["RMSE", "Overall", rolling_overall_metrics['RMSE']],
    ["MAPE", "Overall", rolling_overall_metrics['MAPE']],
    ["MAE", "Peak Hours", rolling_peak_metrics['MAE']],
    ["RMSE", "Peak Hours", rolling_peak_metrics['RMSE']],
    ["MAPE", "Peak Hours", rolling_peak_metrics['MAPE']],
], columns=["Metric", "Type", "Value"])

display(
    rolling_metrics_ref1.style.set_caption("SARIMAX Refinement 1: Rolling Forecast Evaluation")
    .background_gradient(cmap='Blues', subset=["Value"])
)

In [ ]:
# --- 30-Day Future Forecast ---
forecast_future_sarimax = forecast_future_sarimax_model_refined(results_sarimax_best)
forecast_future_sarimax.head()

In [ ]:
# --- Step 11: Future Forecast by Hour ---
future_hourly_avg_df_sr1, future_threshold_sr1, future_peak_hours_sr1 = future_forecast_by_hour_sarimax_refined(forecast_future_sarimax)

# Display heatmap style
display(future_hourly_avg_df_sr1.style.set_caption(" Refined SARIMAX: Future Avg Forecast by Hour").background_gradient(cmap='Blues', axis=0))
print("\nThreshold for Future Peak Hours:", future_threshold_sr1)
print("\nDynamically Selected Future Peak Hours:", future_peak_hours_sr1)

In [ ]:
# --- Step 12: Plot Future Forecast ---
plt.figure(figsize=(12, 6))
plt.plot(forecast_future_sarimax['ds'], forecast_future_sarimax['yhat'], color='blue', label='Forecasted Customer Count')
plt.title("Refined SARIMAX: Future Forecast (Next 30 Days - Hourly)")
plt.xlabel("Date")
plt.ylabel("Forecasted Customer Count")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()